# Notebook for Experimentation and Testing

## Purpose

This notebook was created to follow the instructions from Homework 1, Step 3:
```
Step 3: Set Up Your Playground Notebook

Inside your personal folder, create a Jupyter notebook named playground.ipynb. This notebook will be your dedicated space for experimentation and testing.
```